In [5]:
import requests
import urllib3
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd

In [6]:
clue_a = []
clue_q = []
for i in range(1,6):
    for j in range(1,6):
        clue_a.append('clue_J_'+ str(i) + '_' + str(j) + '_stuck')
        clue_a.append('clue_DJ_'+ str(i) + '_' + str(j) + '_stuck') 
        clue_q.append('clue_J_'+ str(i) + '_' + str(j)) 
        clue_q.append('clue_DJ_' + str(i) + '_' + str(j))

In [41]:
main = []
for i in range(173, 190):
    
    q_url = 'http://www.j-archive.com/showgame.php?game_id=' + str(i)
    a_url = 'http://www.j-archive.com/showgameresponses.php?game_id=' + str(i)
    
    q_status = requests.head(q_url)
    a_status = requests.head(a_url)
 
    if q_status.status_code == 200: 
        if a_status.status_code == 200:
            qres = requests.get(q_url)
            qsoup = BeautifulSoup(qres.content, 'lxml')

            show = qsoup.find('div', {'id':'game_title'}).text
            show_num = re.search('Show #(\d*)', show).group(1)
            show_year = re.search('.*(\d{4})', show).group(1)
            show_day_of_week = re.search('.* - (\w*)\,', show).group(1)
            show_month = re.search('.*, (\w*) \d{1,2}, \d{4}', show).group(1)

            categories = []
            for j in qsoup.find_all('td', {'class':'category_name'}):
                categories.append(j.text)

            clue_questions = []
            clue_values = []
            for j in clue_q:
                clue_exist = qsoup.find('td', {'class':'clue_text', 'id': j})
                if clue_exist:
                    clue_questions.append(qsoup.find('td', {'class':'clue_text', 'id': j}).text)
                    value_exist = qsoup.find('td', {'class':'clue_text', 
                                                         'id': j}).find_next('td',{'class':'clue_value'})
                    if value_exist:
                        clue_values.append(qsoup.find('td', {'class':'clue_text', 
                                                             'id': j}).find_next('td',{'class':'clue_value'}).text)
                    else:
                        clue_values.append('nan')
                else:
                    clue_questions.append('nan')
                    clue_values.append('nan')
            clue_questions.append(qsoup.find('td', {'class':'clue_text', 'id':'clue_FJ'}).text)
            clue_values.append('FJ')

            ares = requests.get(a_url)
            asoup = BeautifulSoup(ares.content, 'lxml')

            answers = []
            for j in clue_a:
                answer_exist = asoup.find('td', {'class':'clue_unstuck', 'id':j})
                if answer_exist: 
                    show = asoup.find('td', {'class':'clue_unstuck', 'id':j})
                    answers.append(show.find_next('em', {'class':'correct_response'}).text)
                else:
                    answers.append('nan')
            answers.append(asoup.find('table', {'class':'final_round'}).find('em',{'class':'correct_response'}).text)

            show_nums, show_months, show_years, show_days = [], [], [], []
            show_cats = []
            for j in range(0,51):
                show_nums.append(show_num)
                show_months.append(show_month)
                show_years.append(show_year)
                show_days.append(show_day_of_week)

                if j <= 4:
                    show_cats.append(categories[0])
                elif j <= 9:
                    show_cats.append(categories[1])
                elif j <= 14:
                    show_cats.append(categories[2])
                elif j <= 19:
                    show_cats.append(categories[3])
                elif j <= 24:
                    show_cats.append(categories[4])
                elif j <= 29:
                    show_cats.append(categories[5])
                elif j <= 34:
                    show_cats.append(categories[6])
                elif j <= 39:
                    show_cats.append(categories[7])
                elif j <= 44:
                    show_cats.append(categories[8])
                elif j <= 49:
                    show_cats.append(categories[9])
                elif j <= 54:
                    show_cats.append(categories[10])
                elif j <= 59:
                    show_cats.append(categories[11])
                elif j <= 64:
                    show_cats.append(categories[12])
                else:
                    show_cats.append(categories[13])


            for j in range(0, 51):
                row = [show_nums[j], show_months[j], show_years[j], show_days[j], show_cats[j], clue_questions[j],
                      clue_values[j], answers[j]]
                main.append(row)
        else:
            pass
    else:
        pass

In [37]:
file = open('jdata.csv', 'w')
with file:
    writer = csv.writer(file)
    writer.writerows(main)

In [38]:
df = pd.read_csv('jdata.csv', header = None)
df.rename(columns = {7:'answer', 4:'category', 5:'clue', 1:'month', 0:'show_number', 3:'day',
                                 6:'value', 2:'year'}, inplace=True)

In [39]:
df.head()

,show_number,month,year,day,category,clue,value,answer
0,1,September,1984,Monday,LAKES & RIVERS,River mentioned most often in the Bible,$100,the Jordan
1,1,September,1984,Monday,LAKES & RIVERS,"When ""Joshua Fit The Battle Of Jericho"", these...",$200,the walls
2,1,September,1984,Monday,LAKES & RIVERS,Scottish word for lake,$200,loch
3,1,September,1984,Monday,LAKES & RIVERS,His price was 30 pieces of silver,$400,Judas
4,1,September,1984,Monday,LAKES & RIVERS,River in this famous song:,$300,the Volga River
